# Scrapping Exercises - Part 2 - 1

Using the selectors from the previous part, we created a spider which get all the information from polytechnique actu page.
We suggest you to run it through this notebook and to have a look to the produced data.

Your next challenge will be to write your own spider.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/NLP_Capgemini_2021/NLP @ X_HEC - 2K21/Midterm")

In [3]:
!pwd
!ls

/content/drive/MyDrive/NLP_Capgemini_2021/NLP @ X_HEC - 2K21/Midterm
Bokan_NLP.ipynb  data  requirements.txt  scraping_main.ipynb


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 245kB 9.7MB/s 
     |████████████████████████████████| 911kB 13.0MB/s 
     |████████████████████████████████| 3.2MB 13.3MB/s 
     |████████████████████████████████| 3.2MB 45.4MB/s 
     |████████████████████████████████| 61kB 11.7MB/s 
     |████████████████████████████████| 3.1MB 37.0MB/s 
     |████████████████████████████████| 245kB 55.6MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=535430e8aebf1d62ce0465784557c29e78730be164773359e398dac26caffa0d
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=afdc4ff6578d201db243f8042474cbe0f8a426cc12a9c9b75a92eda512dca05f
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba67

## 1. Making imports 

In [5]:
# Imports
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logzero
import scrapy

# Set seaborn display
sns.set()

In [6]:
# Parameters
data_directory_path = 'data/'
file_name = 'bokan_ta.jl'

In [7]:
!ls data/

bokan.csv  bokan_ta.jl	newest_gm_reviews.csv


## 3. Reading scrapped data

In [8]:
# Function rto read jl file
def read_jl_file(file_name):
    values = []
    with open(file_name, 'rb') as f:
        line = '---'
        while len(line)>1:
            line = f.readline()
            values.append(line)
    values = values[:-1]
    values = [json.loads(i) for i in values]
    df = pd.DataFrame(values)
    return df

In [15]:
# Reading file
pd.set_option("display.max_rows", None, "display.max_columns", None)
df = read_jl_file(data_directory_path+file_name)
df.sort_values(by='resto', inplace=True)
df.drop_duplicates(inplace=True)

# Having a brief look at data
print(df.shape)

print(df['resto'].value_counts())

(545, 6)
Bokan 37 Restaurant          500
BOKAN 38 Bar & 39 Rooftop     45
Name: resto, dtype: int64


In [16]:
df = df.drop(["review_date", "review_title"], axis=1)
df['source'] = "Tripadvisor"

In [17]:
df = df.reset_index().drop('index', axis=1)
df.head()

,resto,review_rating,review_text,review_link,source
0,BOKAN 38 Bar & 39 Rooftop,2,This place has potential but it has one the mo...,https://www.tripadvisor.co.uk/ShowUserReviews-...,Tripadvisor
1,BOKAN 38 Bar & 39 Rooftop,5,"Wow, where do I start, we had the most amazing...",https://www.tripadvisor.co.uk/ShowUserReviews-...,Tripadvisor
2,BOKAN 38 Bar & 39 Rooftop,5,"Whenever you visit London, Canary Wharf, you j...",https://www.tripadvisor.co.uk/ShowUserReviews-...,Tripadvisor
3,BOKAN 38 Bar & 39 Rooftop,5,"Terrific view, service & atmosphere. The resta...",https://www.tripadvisor.co.uk/ShowUserReviews-...,Tripadvisor
4,BOKAN 38 Bar & 39 Rooftop,2,I am coming very often at Bokan with my friend...,https://www.tripadvisor.co.uk/ShowUserReviews-...,Tripadvisor


In [18]:
df2 = pd.read_csv("data/newest_gm_reviews.csv")

In [19]:
df2['resto'] = "Bokan 37 Restaurant"
df2['source'] = "Google"

df2.columns = ["review_text", "review_rating", "review_link", "resto", "source"]
df2 = df2[["resto", "review_rating",	"review_text",	"review_link",	"source"]]

In [20]:
df = df.append(df2).reset_index().drop("index", axis=1)
df.to_csv("bokan.csv")